In [1]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json
import random

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=4096,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [3]:
client = InferenceClient(
    "", timeout = 40
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

def format_user(history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
    prompt += f"[INST]"
    return prompt

In [4]:
disagree = [
    'bullshit', 'u are lying', 'tipu la ko', 'penipu', 'tipu lah sial', 'ko tipu', 'aku tak caye',
    'wtf tipu', 'ye ke ko ni, tipu lah', 'tipu sial', 'bodoh salah', 'tak betul la sial',
    'tak betul', 'salah', 'ko pasti ke ni', 'tipu tipu tipu',
]

with open('disagree.json') as fopen:
    disagree.extend(json.load(fopen)[:3000])
    
with open('doubt.json') as fopen:
    disagree.extend(json.load(fopen))
    
disagree = [d for d in disagree if len(d) > 5]
len(disagree)

6915

In [5]:
random.choice(disagree)

'Are you as sure as the fact that the universe has a parallel universe?'

In [3]:
from glob import glob

files = sorted(glob('mixtral-disagree-conversation/*.json'))
len(files)

97149

In [7]:
prompts = []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
    questions = []
    splitted = data[1].strip().split('\n')
    splitted = [s for s in splitted if '.' if s and '?' in s]
    splitted = ['.'.join(s.split('.')[1:]).strip() for s in splitted]
    splitted = [s for s in splitted if len(s) > 3]
    splitted = [s[1:] if s[0] == '"' else s for s in splitted]
    splitted = [s[:-1] if s[-1] == '"' else s for s in splitted]
    splitted = [s.strip() for s in splitted]
    questions = [s for s in questions if len(s) > 10]
    questions = sorted(list(set(questions)))
    questions.extend(splitted)
    p = [(data[0], q) for q in questions]
    prompts.extend(p)

len(prompts)

857656

In [8]:
prompts[0]

('Ingat lagi dengan permainan The Sim yang menarik minat ramai pemain diluar sana suatu ketika dahulu? Kini, pembuatnya kembali dengan judul permainan yang baru – Spore. Spore dijangka dikeluarkan pada 7 September 2008 di mana para peminat juga boleh memuat turun permainan ini dari laman Electronic Arts pada 7 September.\n\n\nAntaramuka Permainan Spore\n\nMelalui Spore, pemain boleh mengawal evolusi sesuatu spesis daripada permulaannya – unisel, pembesarannya, dan sehingga ia menjadi suatu makhluk yang berkebolehan dan sosial. Sekiranya anda sambung bermain selepas itu, anda boleh mengawal spesis anda sehingga berjaya ke angkasa lepas, disamping dapat berhubung dengan spesis lain dalam galaksi.\n\nPermainan ini dipecahkan kepada beberapa fasa, dimana disetiap fasa, cara permainan itu berubah-ubah. Sekiranya ketika bermain, spesis anda diserang atau dihancurkan sekali gus, anda akan kembali bermain pada level anda sebelum dihancurkan, atau di fasa sebelum dihancurkan.\n\n\nAntaramuka Pe

In [9]:
from collections import defaultdict

filtered = []
processed = defaultdict(int)
for p in prompts:
    if processed[p[0]] > 3:
        continue
    
    filtered.append(p)
    processed[p[0]] += 1
    
len(filtered)

381055

In [ ]:
!mkdir mixtral-rag-question-disagree
# !rm mixtral-rag-question-disagree/*.json

In [ ]:
def answer(q, i):
    filename = f'mixtral-rag-question-disagree/{i}.json'
    if os.path.exists(filename):
        return
    
    question = f'{q[0]}\n{q[1]}'
    formatted_prompt = format_prompt(question, [])
    while True:
        try:
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            answer = stream.generated_text.strip()
            break
        except:
            pass     
    
    
    history = [(question, answer)]

    for _ in range(random.randint(1, 3)):
        u = random.choice(disagree)
        formatted_prompt = format_prompt(u, history)
        while True:
            try:
                stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
                a = stream.generated_text.strip()
                break
            except Exception as e:
                pass
            
        history.append((u, a))
    
    with open(filename, 'w') as fopen:
        json.dump(history, fopen)

In [ ]:
answer(prompts[0], 0)

In [ ]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [ ]:
from threading import Thread
from queue import Queue

queue = Queue()
urls = [(q, no) for no, q in enumerate(filtered)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 100
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

In [ ]:
for i in range(len(consumers)):
    consumers[i].join()

In [8]:
from glob import glob

files = sorted(glob('mixtral-rag-question-disagree/*.json'))
len(files)

339627

In [19]:
all_texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
        
    all_texts.append(data[0][1])
        
    for d in data[1:]:
        all_texts.extend(d)
    
len(all_texts)

100%|█████████████████████████████████| 339627/339627 [00:35<00:00, 9694.15it/s]


1696183

In [23]:
with open('mixtral-rag-question-disagree.texts', 'w') as fopen:
    for t in set(all_texts):
        if not len(t):
            continue
        fopen.write(f'{json.dumps(t)}\n')

In [24]:
!wc -l mixtral-rag-question-disagree.texts

1027655 mixtral-rag-question-disagree.texts


In [25]:
!cp mixtral-rag-question-disagree.texts ../translation

In [12]:
import re

mapping = {}
for f in glob('/home/husein/ssd3/translation/mixtral-rag-question-disagree.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue

            n = l['r']['result']
            hypens = re.findall('\w+ -\w+', n)
            for h in hypens:
                splitted = h.split('-')
                if len(splitted) != 2:
                    continue
                splitted = [s.strip() for s in splitted]
                splitted = '-'.join(splitted)
                n = n.replace(h, splitted)
            mapping[l['src']] = n
            
len(mapping)

1027455

In [34]:
with open('mixtral-rag-question-disagree.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        with open(f) as fopen:
            data = json.load(fopen)
        splitted = data[0][0].split('\n')
        context = '\n'.join(splitted[:-1])
        q = splitted[-1]
        a = data[0][1]
        chat = [
            {'role': 'user', 'content': q, 'content_ms': mapping.get(q)},
            {'role': 'assistant', 'content': a, 'content_ms': mapping.get(a)}
        ]
        for d in data[1:]:
            chat.extend([
                {'role': 'user', 'content': d[0], 'content_ms': mapping.get(d[0])},
                {'role': 'assistant', 'content': d[1], 'content_ms': mapping.get(d[1])}
            ])
        d = {
            'context': context,
            'chat': chat
        }
        fopen_l.write(f'{json.dumps(d)}\n')

100%|█████████████████████████████████████████████████████████████████████████████████████████| 339627/339627 [00:26<00:00, 12626.24it/s]


In [35]:
!tail -n 1 mixtral-rag-question-disagree.jsonl

{"context": "Mazda US has just issued a recall involving 227,814 units of the Mazda3 and Mazda6 over the possibility of a parking brake failure. We reached out to\u00a0Bermaz Motor (official Mazda distributor in Malaysia) and are pleased to say that Malaysian models are not affected at all.\nAccording to \nNational Highway Traffic Safety Administrations (NHTSA)\n, the parking brake on the affected models have defective seals at the rear brake calliper protective boot which can result in water and debris entering the actuator shaft. This will cause corrosion over time and potentially cause the shaft to be stuck in the brake calliper body. \u00a0\nIn extreme cases, the defected part will \ndecrease the parking brake force\n or to cause brake drag (brakes are engaged even when pedals are not applied) while driving. This poses a safety risk as the \nvehicle may move by itself\n if parked on a slope, even if the parking brake is engaged.\nThe models affected by the recall are Mazda6 assembl

In [36]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='mixtral-malaysian-disagree.jsonl',
    path_in_repo='mixtral-malaysian-disagree.jsonl',
    repo_id='mesolitica/mixtral-malaysian-rag',
    repo_type='dataset',
)

mixtral-malaysian-disagree.jsonl:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/mixtral-malaysian-rag/blob/main/mixtral-malaysian-disagree.jsonl'